In [32]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [60]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [66]:
# remove users who have less than 200 reviews
userNumRatings = df_ratings['user'].value_counts().sort_values(ascending=False)
userNumRatings = userNumRatings[userNumRatings >= 200]
df_ratings = df_ratings[df_ratings['user'].isin(userNumRatings.index)]
df_ratings

,user,isbn,rating
1456,277427,002542730X,10.0
1457,277427,0026217457,0.0
1458,277427,003008685X,8.0
1459,277427,0030615321,0.0
1460,277427,0060002050,0.0
...,...,...,...
1147612,275970,3829021860,0.0
1147613,275970,4770019572,0.0
1147614,275970,896086097,0.0
1147615,275970,9626340762,8.0


In [68]:
# remove books who have less than 100 reviews
bookNumRatings = df_ratings['isbn'].value_counts().sort_values(ascending=False)
bookNumRatings = bookNumRatings[bookNumRatings >= 100]
df_ratings = df_ratings[df_ratings['isbn'].isin(bookNumRatings.index)]
df_ratings

,user,isbn,rating
1469,277427,0060930535,0.0
1471,277427,0060934417,0.0
1474,277427,0061009059,9.0
1495,277427,0142001740,0.0
1513,277427,0312966091,0.0
...,...,...,...
1146823,275970,0440221471,0.0
1146824,275970,0440222656,0.0
1146825,275970,0440226430,0.0
1146852,275970,044651652X,6.0


In [ ]:
# reduce books to just the valid ones
df_books = df_books[df_books['isbn'].isin(df_ratings['isbn'])]
df_books

In [ ]:
# combine book and rating tables
combined_table = pd.merge(df_ratings, df_books, on='isbn')
combined_table

In [ ]:
# new table structure
books_pivot = combined_table.pivot_table(index='title', columns='user', values='rating').fillna(0)
books_pivot

In [77]:
# build model
model = NearestNeighbors(metric='cosine', n_neighbors=5)
model.fit(books_pivot.values)

NearestNeighbors(metric='cosine')

In [80]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  d, i = model.kneighbors([combined_table.loc[book].values], n_neighbors=5)

  recommendations = pd.DataFrame({
      'title'   : combined_table.iloc[i[0]].index.values,
      'distance': d[0]
    })
  recommendations.sort_values(by='distance', ascending=False).head(5).values

  return [book, recommendations]

In [79]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

KeyError: ignored